<a href="https://colab.research.google.com/github/eridanletalis/NLP_rids/blob/main/%D0%9F%D0%B0%D1%80%D1%81%D0%B8%D0%BD%D0%B3_JSON_%D0%B4%D0%BB%D1%8F_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85_%D0%9D%D0%98%D0%9E%D0%9A%D0%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [118]:
import os
import re
import datetime

In [5]:
os.chdir('/content/gdrive/MyDrive/Datasets/niokr') 

In [ ]:
!ls

In [201]:
with open('rid_2016_05.json', 'r') as file:
  data = json.load(file)

In [98]:
def parse_organizations(org: dict) -> str:
    """
      short_name
      name
      original_name
      original_short_name
      ogrn
      supervisor_name
      supervisor_surname
      supervisor_patronymic
      organization_type
      territory
      okogu
      okopf
    """
    if org['name']:
        org_name = org['name']
    elif org['short_name']:
        org_name = org['short_name']
    else:
        org_name = ''

    if org['original_name']:
        original_name = org['original_name']
    elif org['original_short_name']:
        original_name = org['original_short_name']
    else:
        original_name = ''

    if org['okogu']:
        okogu = org['okogu']
    else:
        okogu = ''

    if org['ogrn']:
        ogrn = f"ОГРН: {org['ogrn']}"
    else:
        ogrn = ''
    
    return f'{org_name} {original_name} {ogrn} {okogu}'.strip()

# print(parse_organizations(lst))

Федеральное государственное бюджетное образовательное учреждение высшего образования  "Курский государственный медицинский университет" Министерства здравоохранения  Российской Федерации  ОГРН: 1034637005347 Министерство здравоохранения Российской Федерации


In [85]:
def parse_supervisors(spec: dict) -> str:
  result: str = ''
  for l in spec:

    if l['responsible_type']:
      restp_type = l['responsible_type']
    else:
      restp_type = 'Научный руководитель'

    if l['scientific_degree']:
      s_degree = l['scientific_degree']
      s_degree += ', '
    else:
      s_degree = ' '
  
    if l['fio']:
      fio = l['fio']
      result += ' '
    else:
      fio = 'Неизвестен.'

    if l['speciality_code']:
      speciality_code = l['speciality_code']
    else:
      speciality_code = ' неизветна'

    result += f'{restp_type}: {s_degree} {fio} Специализация: {speciality_code}'
    result += '; '
  return result.strip()


# print(parse_supervisors(lst))


Научный руководитель: Доктор технических наук,  Луганцев Л.Д. Специализация: Машины, агрегаты и процессы (по отраслям);


In [68]:
def parse_speciality_codes(spec: dict) -> str:
  result: str = ''
  for l in spec:
    for key in l:
      if l[key]:
        result += l[key]
        if key == 'name':
          result += ', '

    result += '; '
  return result.strip()


# print(parse_speciality_codes(lst))


Клеточная биология, цитология, гистология, 03.03.04; Акушерство и гинекология, 14.01.01;


In [72]:
def parse_dissertation_author_name(surname: str, name=None) -> str:
  autor_name_result = ''
  if data['cards'][i]['author_surname']:
    autor_name_result += data['cards'][i]['author_surname']
  autor_name_result += ' '
  if data['cards'][i]['author_name']:
    autor_name_result += data['cards'][i]['author_name']
  return autor_name_result.strip()

In [212]:
df = pd.DataFrame(columns=list(data['cards'][0]))

In [227]:
def parse_executors(lst: list) -> str:
  exct = ''
  for l in lst:
    exct += parse_organizations(l)
  exct += '; '

  return exct.strip()

In [181]:
def parse_ikrbs_authors(lst: list) -> str:
  authors = ''

  for pers in lst:
    if pers['degree']:
      degree = pers['degree']
    else:
      degree = ''

    if pers['position']:
      pos = pers['position']
    else:
      pos = ''
    
    if pers['rank']:
      rank = pers['rank']
    else:
      rank = ''
    
    if pers['description']:
      descr = pers['description']
    else:
      descr = ''

    if pers['surname']:
      surname = pers['surname']
    else:
      surname = 'NA'

    if pers['name']:
      name = pers['name']
    else:
      name = 'NA'     

    if pers['patronymic']:
      patr = pers['patronymic']
    else:
      patr = 'NA'

    authors += f'{descr} {surname} {name} {patr} {pos} {rank} {degree} '.strip()
    authors += '; '

  return authors.strip()

In [169]:
def parse_ikrbs_supervisor(pers: dict) -> str:
  if pers['degree']:
    degree = pers['degree']
  else:
    degree = ''

  if pers['position']:
    pos = pers['position']
  else:
    pos = ''
  
  if pers['rank']:
    rank = pers['rank']
  else:
    rank = ''
  
  if pers['surname']:
    name = pers['surname']
  else:
    'Неизвестное имя'

  return f'{pos} {rank} {degree} {name}'.strip()

  

In [156]:
def parse_ikrbs_publications(lst: list) -> str:
  publications = ''
  for l in lst:
    if l['authors']:
      atr = ''
      for i in l['authors']:
        if i['name']:
          atr += i['name']
          atr += ' '
        if i['surname']:
          atr += i['surname']
          atr += ' '

    if l['publication_date']:
      pbl_date = l['publication_date']
    else:
      pbl_date = ''
    if l['publication_name']:
      pbl_name = l['publication_name']
    else:
      pbl_name = ''
    if l['edition_name']:
      pbl_ed_name = l['edition_name']
    if l['edition_type']:
      pbl_type = l['edition_type']
    else:
      pbl_type = ''

    publications += f'{pbl_type} {atr.strip()} {pbl_name} {pbl_ed_name}'.strip()
    publications += '; '

  return publications.strip()

In [233]:
for i in range(len(data['cards'])):

  try:
    print(data['cards'][i]['name'])
  except:
    print('Работа никак не озаглавлена')

# Обработка результатов НИОкров
  if file.name.find('ikrbs') == 0:
    try:
      print(datetime.datetime.strptime(data['cards'][i]['approve_date'], 
                                 '%Y-%m-%d'))
      
    except:
      date = str(re.search('\d\d\d\d_\d\d', file.name)[0]) + '_01' 
      print(datetime.datetime.strptime(date, '%Y_%m_%d'))

    try:
      print(parse_ikrbs_authors(data['cards'][i]['authors']))
    except:
      print('Нет информации об авторах работы')

    try:
      print(parse_ikrbs_supervisor(data['cards'][i]['work_supervisor']))
    except:
      print('Нет информации о научном руководителе')

    try:
      print(parse_ikrbs_supervisor(
          data['cards'][i]['organization_supervisor']))
    except:
      print('Нет информации об ответственном от организации')

    try:
      print(parse_ikrbs_publications(data['cards'][i]['publications']))
    except:
      print('Нет информации о публикациях')

    try:
      print(data['cards'][i]['executor'])
    except:
      print('Нет информации об исполнителе. Нужно искать в начинаемых НИОКТРах')

    try:
      print(data['cards'][i]['nioktr'])
    except:
      print('Нет информации о связанных НИОКТР')


# Обработка отчётов о РИДах
  elif file.name.find('rid') == 0:

    try:
      print(datetime.datetime.strptime(data['cards'][i]['created_date'], 
                                 '%Y-%m-%d'))
      
    except:
      date = str(re.search('\d\d\d\d_\d\d', file.name)[0]) + '_01' 
      print(datetime.datetime.strptime(date, '%Y_%m_%d'))
    
    try:
      print(parse_ikrbs_authors(data['cards'][i]['authors']))
    except:
      print('Нет сведений об авторах')

    try:
      print(parse_executors(data['cards'][i]['executors']))
    except:
      print('Нет информации об исполнителях')

    try:  
      print(parse_ikrbs_supervisor(data['cards'][i]['work_supervisor']))
    print('Нет информации о научном руководителе')
    
# Обработка диссертаций    
  elif file.name.find('dissertation') == 0:
    try:
      print(datetime.datetime.strptime(data['cards'][i]['protection_date'], 
                                 '%Y-%m-%d'))
    except:
      date = str(re.search('\d\d\d\d_\d\d', file.name)[0]) + '_01' 
      print(datetime.datetime.strptime(date, '%Y_%m_%d'))

    try:
      print(parse_dissertation_author_name(data['cards'][i]['author_surname'], 
                                          data['cards'][i]['author_name']))
    except:
      print('Произошла ошибка поиска имени автора')
    
    try:
      print(parse_supervisors(data['cards'][i]['supervisors']))
    except:
      print('Нет данных о руководителе')

    try:
      print(parse_organizations(data['cards'][i]['author_organization']))
    except:
      print('Нет данных об организации автора')

    try:  
      print(parse_organizations(data['cards'][i]['protection_organization']))
    except:
      print('Нет данных об организации, где происходила защита')

    try:
      print(parse_speciality_codes(data['cards'][i]['speciality_codes']))
    except:
      print('Нет данных о кодах специализации')

  try:
  # kw = [kw.strip() for kw in data['cards'][i]['keyword_list'][0].split(', ')]
    kw = [kw.strip() for kw in re.split(r'[",;"]', data['cards'][i]['keyword_list'][0])]
    print(', '.join(kw))
  except:
    try:
      print(data['cards'][i]['keyword_list'])
    except:
      print('Нет данных о ключевых словах')

  try:
    print(data['cards'][i]['abstract'])
  except:
    print('Нет данных об аннотации')

  print('*'*50)



Рекомендации по научному развитию системы антидотного обеспечения Российской Федерации
2016-05-10 00:00:00
Нормы обеспечения субъектов Российской Федерации и учреждений федерального уровняантидотами; Новиков Сергей Викторович; Состояние и перспективные направления развития системы антидотной терапии; Самойлов Александр Сергеевич; Материалы и методы исследований; Баландин Никита Викторович; Рекомендации по научному развитию системы антидотного обеспечения РоссийскойФедерации; Бояринцев Валерий Владимирович; Программа производственного развития системы антидотного обеспечения в РоссийскойФедерации; Дружков Алексей Вячеславович; Анализ существующих угроз чрезвычайных ситуаций химической природы Беловолов Антон Юрьевич;
Федеральное государственное унитарное предприятие Научно-производственный центр "Фармзащита" Федерального медико-биологического агентства  ОГРН: 1035009556856 Федеральное медико-биологическое агентство;
Зам. директора по науке   Гладких В.Д.
АНТИДОТНАЯ ТЕРАПИЯ, ЛЕЧЕБНО-ПРОФ

In [186]:
df.to_excel("output.xlsx", sheet_name='Sheet_name_1')

In [ ]:
df = pd.read_json('nioktr_2020_01.json')